## Introduction

In this project we will work with housing data for the city of Ames, Iowa, United States from 2006 to 2010.

In [1]:
# import pandas, matplotlib & numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import scikit-learn classes
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [2]:
# read file
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")

In [3]:
def transform_features(df):
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):
    train = df[:1460]
    test= df[1460:]
    
    # Select only the numerical columns from train and test
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    #Select all featires except 'SalePrice'
    features = numeric_train.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

transform_df = transform_features(df)
filter_features = select_features(transform_df)
rmse = train_and_test(filter_features)

print(rmse)

57088.25161263909


## Feature improvements

- Missing value
    - For all Columns
        - Drop any columns have 5% or more missing values
    - For text columns
        - Drop with 1 or more missing values
    - For Numercial columns
        - Columns with missing values, fill with most common values

In [4]:
## Drop columns with 5% or more missing data

# Summerize all missing data
num_missings = df.isnull().sum()

# Filter 5% or more missing data columns
drop_missing_columns = num_missings[(num_missings > len(df)/20)].sort_values()

# Drop the Filtered columns
df= df.drop(drop_missing_columns.index, axis=1)
print(df.isnull().sum())

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Area            0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Type       23
Mas Vnr Area       23
Exter Qual          0
Exter Cond          0
Foundation          0
Bsmt Qual          80
                   ..
Electrical          1
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Bsmt Full Bath      2
Bsmt Half Bath      2
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Fireplaces

### Drop any 1 or more missing values from text columns

In [5]:
# missing values for series objects
text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)

# Filter missing values 1 or more
drop_missing_columns2 = text_mv_counts[text_mv_counts > 0]

df=df.drop(drop_missing_columns2.index, axis=1)
print(df.isnull().sum())

Order               0
PID                 0
MS SubClass         0
MS Zoning           0
Lot Area            0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Area       23
Exter Qual          0
Exter Cond          0
Foundation          0
BsmtFin SF 1        1
BsmtFin SF 2        1
                   ..
Central Air         0
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Bsmt Full Bath      2
Bsmt Half Bath      2
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qual        0
TotRms AbvGrd       0
Functional          0
Fireplaces

### Fill up the missing numerical columns with most common value

In [6]:
# Summary of numeric columns
num_colum = df.select_dtypes(include=['int', 'float']).isnull().sum()

# Numerical columns with missing values
num_missing = num_colum[num_colum > 0].sort_values()
num_missing

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [7]:
# Compute most comon values for each missing columns
common_values_dict = df[num_missing.index].mode().to_dict(orient='records')[0]
common_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [8]:
## Fill the missing date
df = df.fillna (common_values_dict)
print(df.isnull().sum().value_counts())

0    64
dtype: int64


Create new features with the existing feature for better information

In [9]:
years_old = df['Yr Sold'] - df['Year Built']
# years_old[years_old <0]
years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
# years_since_remod[years_since_remod < 0]


# Create new columns
df['Years before sale'] = years_old
df['Years since Remod'] = years_since_remod

neg_rows = df[(df['Years before sale'] < 0) | (df['Years since Remod'] < 0)].index
df= df.drop(neg_rows, axis=0)

## No longer need original year columns
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)


- Drop columns that are 
    - not useful for ML
    - Have data leakage



In [10]:
## Drop columns that aren't useful for ML
df = df.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

* Update transform_features() fuction which contains all the above modification

In [11]:
def transform_features(df):
    num_missings = df.isnull().sum()
    drop_missing_columns = num_missings[(num_missings > len(df)/20)].sort_values()
    df= df.drop(drop_missing_columns.index, axis=1)

    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_columns2 = text_mv_counts[text_mv_counts > 0]
    df=df.drop(drop_missing_columns2.index, axis=1)

    num_colum = df.select_dtypes(include=['int', 'float']).isnull().sum()
    num_missing = num_colum[num_colum > 0].sort_values()
    common_values_dict = df[num_missing.index].mode().to_dict(orient='records')[0]
    df = df.fillna (common_values_dict)

    years_old = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years before sale'] = years_old
    df['Years since Remod'] = years_since_remod

    neg_rows = df[(df['Years before sale'] < 0) | (df['Years since Remod'] < 0)].index
    df= df.drop(neg_rows, axis=0)
    df = df.drop([], axis=1)
    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Yr Sold", "Year Built", "Year Remod/Add"], axis=1)
    
    return df

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

def train_and_test(df):
    train = df[:1460]
    test= df[1460:]
    
    # Select only the numerical columns from train and test
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    #Select all featires except 'SalePrice'
    features = numeric_train.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    
    return rmse

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filter_features = select_features(transform_df)
rmse = train_and_test(filter_features)

rmse

55275.36731241307

## Feature Selction for Numerical featires

In [12]:
numerical_df = transform_df.select_dtypes(include = ['int', 'float'])
numerical_df.head(5)

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years before sale,Years since Remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,210,62,0,0,0,0,0,215000,50,50
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,140,0,0,0,120,0,0,105000,49,49
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,393,36,0,0,0,0,12500,172000,52,52
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,0,0,0,0,0,0,0,244000,42,42
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,212,34,0,0,0,0,0,189900,13,12


In [13]:
abs_corr_coeff = numerical_df.corr()['SalePrice'].abs().sort_values()

abs_corr_coeff[abs_corr_coeff > .4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years since Remod    0.534985
Full Bath            0.546118
Years before sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [14]:
# Drop columns with correlation coefficients less than 0.4
transform_df = transform_df.drop (abs_corr_coeff[abs_corr_coeff > .4].index, axis=1)


#### Selecting categorical columns

In [15]:
# List of columns name as categorical selected from documentation
nominal_variables = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", "Misc Feature", "Sale Type", "Sale Condition"]

In [16]:
# Short out the catagorical columns that are already in our data set
select_cata_cols = []

for col in nominal_variables:
    if col in transform_df.columns:
        select_cata_cols.append(col)
        
# Check unique values of selected catagorical columns
unique_count = transform_df[select_cata_cols].apply(lambda col: len(col.value_counts())).sort_values()

# Cut off value more than 10
drop_nonuniq_cols = unique_count[unique_count > 10].index
drop_nonuniq_cols
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

In [17]:
## Select just the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
    
## Create dummy columns and add back to the dataframe!
transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1).drop(text_cols,axis=1)

#### Update select_features()

In [33]:
def transform_features(df):
    num_missings = df.isnull().sum()
    drop_missing_columns = num_missings[(num_missings > len(df)/20)].sort_values()
    df= df.drop(drop_missing_columns.index, axis=1)

    text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
    drop_missing_columns2 = text_mv_counts[text_mv_counts > 0]
    df=df.drop(drop_missing_columns2.index, axis=1)

    num_colum = df.select_dtypes(include=['int', 'float']).isnull().sum()
    num_missing = num_colum[num_colum > 0].sort_values()
    common_values_dict = df[num_missing.index].mode().to_dict(orient='records')[0]
    df = df.fillna (common_values_dict)

    years_old = df['Yr Sold'] - df['Year Built']
    years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
    df['Years before sale'] = years_old
    df['Years since Remod'] = years_since_remod

    neg_rows = df[(df['Years before sale'] < 0) | (df['Years since Remod'] < 0)].index
    df= df.drop(neg_rows, axis=0)
    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Yr Sold", "Year Built", "Year Remod/Add"], axis=1)
    
    return df

def select_features(df, coeff_th=0.4, uniq_th=10):
    numerical_df = df.select_dtypes(include = ['int', 'float'])
    abs_corr_coeff = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop (abs_corr_coeff[abs_corr_coeff < coeff_th].index, axis=1)
    # print(df['SalePrice'])
    
    nominal_variables = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                         "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                         "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", "Misc Feature", 
                         "Sale Type", "Sale Condition"]
    
    
    select_cata_cols = []
    for col in nominal_variables:
        if col in df.columns:
            select_cata_cols.append(col)     

    unique_count = df[select_cata_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = unique_count[unique_count > uniq_th].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1).drop(text_cols,axis=1)   
    
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()

    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)
    
        return rmse

    if k == 1:
        shuffle_df = df.sample(frac=1)
        fold_one = shuffle_df[:1460]
        fold_two = shuffle_df[1460:]
        
        lr.fit(fold_one[features], fold_one["SalePrice"])
        predictions_one = lr.predict(fold_two[features])
        
        mse_one = mean_squared_error(fold_two["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(fold_two[features], fold_two["SalePrice"])
        predictions_two = lr.predict(fold_one[features])
        
        mse_two = mean_squared_error(fold_one["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        
        print(rmse_one)
        print(rmse_two)
           
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse
        
        

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filter_features = select_features(transform_df)
rmse = train_and_test(filter_features, k=4)

rmse

[28219.47959384637, 36618.556937611436, 27070.570914961518, 25136.176529373908]


29261.195993948306